In [ ]:
import MetaTrader5 as mt
import time
import numpy as np
from datetime import datetime

# Initialize MetaTrader5
mt.initialize()

# Define trading parameters
SYMBOL = "EURUSD"
VOLUME = 0.01  # Lot size
DEVIATION = 20  # Allowed price deviation
MAGIC = 234000  # Magic number for identifying orders
TIMEFRAME = 3600  # Execute every hour (3600 seconds)
PIP = 0.0001  # 1 pip for most Forex pairs
SL_PIPS = 60 * PIP  # 60 pips stop-loss
TP_PIPS = 600 * PIP  # 600 pips take-profit

# Function to get open position
def get_open_position(symbol):
    positions = mt.positions_get(symbol=symbol)
    if positions:
        return positions[0]  # Return the first open position
    return None

# Function to close an open trade
def close_trade(position):
    request = {
        "action": mt.TRADE_ACTION_DEAL,
        "symbol": SYMBOL,
        "volume": position.volume,
        "type": mt.ORDER_TYPE_BUY if position.type == mt.ORDER_TYPE_SELL else mt.ORDER_TYPE_SELL,
        "price": mt.symbol_info_tick(SYMBOL).ask if position.type == mt.ORDER_TYPE_SELL else mt.symbol_info_tick(SYMBOL).bid,
        "deviation": DEVIATION,
        "magic": MAGIC,
        "comment": "Auto close order",
        "type_time": mt.ORDER_TIME_GTC,
        "type_filling": mt.ORDER_FILLING_IOC,
    }
    result = mt.order_send(request)
    print(f"Closed order: {result}")

# Function to send a new trade order
def send_order(signal):
    position = get_open_position(SYMBOL)
    
    if position:
        # Close existing trade if signal changes
        if (signal == "buy" and position.type == mt.ORDER_TYPE_SELL) or \
           (signal == "sell" and position.type == mt.ORDER_TYPE_BUY):
            close_trade(position)

    # If no open position, place a new order
    if not get_open_position(SYMBOL):
        price = mt.symbol_info_tick(SYMBOL).ask if signal == "buy" else mt.symbol_info_tick(SYMBOL).bid
        sl = price - SL_PIPS if signal == "buy" else price + SL_PIPS
        tp = price + TP_PIPS if signal == "buy" else price - TP_PIPS

        request = {
            "action": mt.TRADE_ACTION_DEAL,
            "symbol": SYMBOL,
            "volume": VOLUME,
            "type": mt.ORDER_TYPE_BUY if signal == "buy" else mt.ORDER_TYPE_SELL,
            "price": price,
            "sl": sl,
            "tp": tp,
            "deviation": DEVIATION,
            "magic": MAGIC,
            "comment": "Auto trade",
            "type_time": mt.ORDER_TIME_GTC,
            "type_filling": mt.ORDER_FILLING_IOC,
        }
        result = mt.order_send(request)
        print(f"New order sent: {result}")

# Function to get current trading signal from model
def get_trading_signal(y_pred):
    y_pred_classes = np.argmax(y_pred, axis=1)
    last_signal = y_pred_classes[-1]  # Get the last predicted value
    if last_signal == 1:
        return "sell"
    elif last_signal == 2:
        return "buy"
    return "hold"

# Function to check market open hours (Forex: Monday to Friday)
def is_market_open():
    now = datetime.utcnow()
    return now.weekday() < 5  # Monday (0) to Friday (4)

# Main loop to run hourly
while True:
    if is_market_open():
        y_pred = np.random.rand(10, 3)  # Replace with your model's prediction
        signal = get_trading_signal(y_pred)
        
        if signal in ["buy", "sell"]:
            send_order(signal)
        else:
            print("Holding position, no trade executed.")

    # Sleep for 1 hour before next execution
    time.sleep(TIMEFRAME)

# Shutdown MetaTrader5 connection
mt.shutdown()
